In [2]:
import pandas
import numpy

# need this for using pandas built-in plotting facility
import matplotlib.pyplot as plt
%matplotlib inline

# please visit 'http://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#sphx-glr-auto-examples-preprocessing-plot-all-scaling-py'
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingClassifier


pandas.set_option('display.max_rows', 10)
pandas.set_option('display.max_columns', 10)

# set a fixed seed for numpy pseudo random generator
numpy.random.seed(100)

In [ ]:
data = pandas.read_excel("./datasets/Online Retail.xlsx")

In [ ]:
data